In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re

my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}

#古騰堡首頁連線
url = 'https://www.gutenberg.org/browse/languages/zh'
res = req.get(url, headers=my_headers)
soup = bs(res.text, 'lxml')

In [2]:
#從首頁抓取每本小說的頁面url
novelURLs = tuple(soup.select('li.pgdbetext a'))

#計數器，到250結束
counter = 0
#迴圈準備進小說頁面
for i in novelURLs:
    
    #進小說頁面
    novelName = i.get_text()
    link = 'https://www.gutenberg.org'+i['href']
    res_ = req.get(link, headers=my_headers)
    soup_ = bs(res_.text, 'lxml')
    print(f'準備下載: {novelName}，連結: {link} 。', end='')
    
    #確認小說語言，英文小說跳過不抓
    En = False
    for lang in soup_.select('tr[property="dcterms:language"] td'):
        if lang.get_text()=='English':
            En = True
    if En:
        print(f'{novelName}為英文小說，跳過')
        continue

    #進入內文頁面
    text_link = 'https://www.gutenberg.org'+soup_.select_one('a[type="text/html"]')['href']
    res_ = req.get(text_link, headers=my_headers)
    soup_ = bs(res_.text, 'lxml')
    print(f'進入內文頁面，開始解析。', end='')
    
    
    #抓取內文，刪掉英文部分(標題中文字會被保留導致重複)
    text = soup_.select_one('body').get_text()
    pattern = r"(?=[^。!？:；)」』》〉～~、．…])[\s\W\dA-Za-z]+(?<=[A-Za-z])(?=[^A-Za-z])[\.\!\:\s]*"
    text = re.sub(pattern, '', text)
    
    #去除開頭重複兩次的標題
    novelName = soup_.select_one('body section.pg-boilerplate.pgheader h2 span').get_text()
    text = re.sub(novelName, '', text, 2)
    
    #標題到內文中間加換行
    text = f'標題: <{novelName}>' +'\n'+ text
    
    #寫入text
    path = f"作業-小說下載/{novelName}.txt"
    with open(path, 'w',encoding='utf-8-sig') as f:
        f.write(text)
    print(f'{novelName}下載完成，目前共下載{counter+1}/250本小說')

    #計數器+1，到250時結束
    counter+=1
    if counter > 250:
        print('結束下載')
        break

準備下載: 豆棚閒話，連結: https://www.gutenberg.org/ebooks/25328 。進入內文頁面，開始解析。豆棚閒話下載完成，目前共下載1/250本小說
準備下載: 戲中戲，連結: https://www.gutenberg.org/ebooks/24225 。進入內文頁面，開始解析。戲中戲下載完成，目前共下載2/250本小說
準備下載: 比目魚，連結: https://www.gutenberg.org/ebooks/27119 。進入內文頁面，開始解析。比目魚下載完成，目前共下載3/250本小說
準備下載: 比目魚，連結: https://www.gutenberg.org/ebooks/24185 。進入內文頁面，開始解析。比目魚下載完成，目前共下載4/250本小說
準備下載: Study of Inner Cultivation，連結: https://www.gutenberg.org/ebooks/38585 。Study of Inner Cultivation為英文小說，跳過
準備下載: 三字經，連結: https://www.gutenberg.org/ebooks/12479 。進入內文頁面，開始解析。三字經下載完成，目前共下載5/250本小說
準備下載: 山水情，連結: https://www.gutenberg.org/ebooks/25146 。進入內文頁面，開始解析。山水情下載完成，目前共下載6/250本小說
準備下載: 山海經，連結: https://www.gutenberg.org/ebooks/25288 。進入內文頁面，開始解析。山海經下載完成，目前共下載7/250本小說
準備下載: 施公案，連結: https://www.gutenberg.org/ebooks/23825 。進入內文頁面，開始解析。施公案下載完成，目前共下載8/250本小說
準備下載: 施公案，連結: https://www.gutenberg.org/ebooks/25393 。


KeyboardInterrupt

